In [10]:
# Import Librerie
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

config = {}
config_path = os.path.join(os.path.abspath('../../'))
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)

In [11]:
# get database connection
db=config['DATABASE_ELE']
schema=config['SCHEMA_ELE']
engine = create_engine(db)

user1 = config['USER1']
user2 = config['USER2']
user3 = config['USER3']

In [8]:
#Leggo il file messo a disposizione da Istat per i comuni
url="http://www.istat.it/storage/codici-unita-amministrative/Elenco-comuni-italiani.csv"
df_comuni = pd.read_csv(url,sep = ';',encoding='latin-1')

In [ ]:
# get today's date
todays_date = datetime.datetime.now()

In [26]:
# Leggo le ultime news
cur = engine.execute(
    '''     
    SELECT
    "desc" as msg
    ,to_char ("pubAt"::timestamp at time zone 'UTC', 'YYYY-MM-DD') as dt
    ,fonte as fonte
    ,"user" as user
    FROM ''' + schema + '''."news"
    WHERE dt_rif=(select max(dt_rif) from ''' + schema + '''."news")
    ''')
f_news = cur.fetchall()
header = ['msg','dt','fonte','user']
df_news = pd.DataFrame(f_news, columns=header)
df_news.head(2)

,msg,dt,fonte,user
0,Tra gli altri ospiti della puntata del 3 dicem...,2017-12-03,Sorrisi.com,Renzi
1,Giacomo Amadori per “la Verità” il servizio di...,2017-12-03,Dagospia.com,Renzi


In [34]:
# Leggo gli ultime social
cur = engine.execute(
    '''     
    SELECT
    msg as msg
    ,to_char(dt::timestamp, 'YYYY-MM-DD') as dt
    ,sorgente as fonte
    ,"user" as user
    FROM ''' + schema + '''."timeline"
    WHERE 
    sorgente in ('twitter','facebook') and
    dt_rif=(select max(dt_rif) from ''' + schema + '''."timeline")
    ''')
f_social = cur.fetchall()
header = ['msg','dt','fonte','user']
df_social = pd.DataFrame(f_social, columns=header)
df_social.head(2)

,msg,dt,fonte,user
0,Questo video è girato ovunque. È un falso: mai...,2017-12-03,twitter,Renzi
1,RT @GianluigiNuzzi: La buona domenica e’ che N...,2017-12-03,twitter,Di Maio
